# ДЗ 1 Индекс

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзей. Одна серия - один документ.

**To do:** 

**1 Создайте обратный индекс этой базы в двух форматах: словарь и матрица.**

Компоненты вашей реализации:
    - Функция препроцессинга данных. Включите туда лемматизацию, приведение к одному регистру, удаление пунктуации и стоп-слов.
    - Функция индексирования данных. На выходе создает обратный индекс.

**2 С помощью обратного индекса в каждом формате посчитайте:** 


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) кто из главных героев статистически самый популярный (упонимается чаще всего)? Имена героев:
- Моника, Мон
- Рэйчел, Рейч
- Чендлер, Чэндлер, Чен
- Фиби, Фибс
- Росс
- Джоуи, Джои, Джо

**На что направлены эти задачи:** 
1. Навык построения обратного индекса
2. Навык работы с этим индексом

[download_friends_corpus](https://yadi.sk/d/4wmU7R8JL-k_RA?w=1)

imports

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 9.1 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ca52e4c5944c968355e2277f5b933bb66bf65d672f5e15d4405169409fff75cd
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [31]:
import re
import nltk
from nltk.corpus import stopwords
import pymorphy2
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

getting data

In [13]:
path = '/content/drive/MyDrive/infosearch22/hw1/friends-data'

In [18]:
corpus = []
for root, dirs, files in os.walk(path):
    for name in files:
        with open(os.path.join(root, name)) as f:
          text = f.read()
          corpus.append(text)

preprocessing

In [23]:
nltk.download('stopwords')
stopwords = stopwords.words("russian")#stop words
morph = pymorphy2.MorphAnalyzer() #lemmatizer

TOKEN_RE = re.compile(r'[а-яё]+')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
def preprocess(text):
    text = ''.join(text.split('\n')[:-4]) # remove metainfo
    text = text.lower() #lower case
    text = re.sub(r'[^\w\s]', '', text) #punktuation
    drt_tokens = text.split() # creating a list of tokens
    tokens = [] # empty list for proceed tokens
    for i in drt_tokens:
      if i.isalpha() and i not in stopwords:
        i = morph.normal_forms(i.strip())[0]
        tokens.append(i)
    return tokens

number of texts (episodes)

In [24]:
# в коллекции должно быть 165 файлов

len(corpus)

165

In [27]:
prep_texts = [preprocess(x) for x in corpus]

In [28]:
len(prep_texts)

165

check the results of preprocessing on the first text, first 10 tokens:

In [30]:
prep_texts[0][:10]

['фибиести',
 'всемогущий',
 'денья',
 'пожелать',
 'мир',
 'весь',
 'мирей',
 'голодая',
 'беречь',
 'тропический']

In [34]:
# list of names:

names = [['Моника', 'Мон'],
         ['Рэйчел', 'Рейч', 'Рэйч'],
         ['Чендлер', 'Чэндлер', 'Чен'],
         ['Фиби', 'Фибс'],
         ['Росс'],
         ['Джоуи', 'Джои', 'Джо']]

# get lowercase:

NAMES = []
for n in names:
  l = []
  for nn in n:
    l.append(nn.lower())
  NAMES.append(l)

NAMES

[['моника', 'мон'],
 ['рэйчел', 'рейч', 'рэйч'],
 ['чендлер', 'чэндлер', 'чен'],
 ['фиби', 'фибс'],
 ['росс'],
 ['джоуи', 'джои', 'джо']]

In [45]:
def matrix_case(prep_texts):  
  vectorizer = CountVectorizer(analyzer='word')
  X = vectorizer.fit_transform(prep_texts)

  features = vectorizer.get_feature_names()
  matrix_freq = np.asarray(X.sum(axis=0)).ravel()
  most_freq = features[np.argmax(matrix_freq)]
  less_freq = features[np.argmin(matrix_freq)]

  full = np.apply_along_axis(lambda x: 0 not in x, 0, X.toarray())
  id = np.where(full)[0]
  every_word = [features[i] for i in id]
  names_ = {}

  for char in NAMES:
      count = 0
      for name in char:
          index = vectorizer.vocabulary_.get(name.lower())
          if index:
              count += X.T[index].sum()
      names_[char[0]] = 0
      names_[char[0]] += count

  freq_char = sorted(names_.items(), key=lambda x: x[1], reverse=True)[0][0]

  return most_freq, less_freq, every_word, freq_char

In [36]:
def create_dictionary(texts):
    dictionary = {}
    for keys, values in texts.items():
        for i in values:
            rvr = dictionary.setdefault(i, {})
            rvr[keys] = rvr.get(keys, 0) + 1
    return dictionary

In [37]:
def dict_case(d):
    rotated_dic = create_dictionary(d)

    counts = {} 
    for word, documents in rotated_dic.items():
        for k, v in documents.items():
            counts[word] = counts.get(word, 0) + v

    sorted_dict = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
    most_freq_d = list(sorted_dict.items())[0][0]
    less_freq_d = list(sorted_dict.items())[-1][0]
    every_word_d = [word for word, documents in rotated_dic.items() if len(documents) == 165]

    chars = {}
    for c in NAMES:
        counter = 0
        for name in c:
            try:
                counter += counts[name]
            except KeyError:
                pass
        chars[c[0]] = 0
        chars[c[0]] += counter
    freq_char_d = sorted(chars.items(), key=lambda x: x[1], reverse=True)[0][0]

    return most_freq_d, less_freq_d, every_word_d, freq_char_d

In [50]:
d = {}
i = 1

prep_corp = [' '.join(x) for x in prep_texts]

for p in range(len(prep_corp)):
  d[p + 1] = prep_corp[p]


most_freq, less_freq, every_doc, most_freq_char = matrix_case(prep_corp)
most_freq_d, least_freq_d, every_doc_d, most_freq_char_d = dict_case(d)

print('\n===== RESULTS FOR MATRIX APPROACH: =====\n')
print('Most frequent word in the matrix: {}'.format(most_freq))
print('Least frequent word in the matrix: {}'.format(less_freq))
print('Word that appears in every document: {}'.format(every_doc))
print('Most frequently mentioned character: {}'.format(most_freq_char.capitalize()))
print('\n===== RESULTS FOR DICTIONARY APPROACH: =====\n')
print('Most frequent word in the matrix: {}'.format(most_freq_d))
print('Least frequent word in the matrix: {}'.format(least_freq_d))
print('Word that appears in every document: {}'.format(every_doc_d))
print('Most frequently mentioned character: {}'.format(most_freq_char_d.capitalize()))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



===== RESULTS FOR MATRIX APPROACH: =====

Most frequent word in the matrix: это
Least frequent word in the matrix: after
Word that appears in every document: ['знать', 'это']
Most frequently mentioned character: Росс

===== RESULTS FOR DICTIONARY APPROACH: =====

Most frequent word in the matrix:  
Least frequent word in the matrix: x
Word that appears in every document: ['ф', 'и', 'б', 'е', 'с', 'т', ' ', 'в', 'м', 'о', 'г', 'у', 'щ', 'й', 'д', 'н', 'ь', 'я', 'п', 'ж', 'л', 'а', 'р', 'ч', 'к', 'ш', 'з', 'ы', 'ё', 'э', 'х', 'ю', 'ц']
Most frequently mentioned character: Моника
